<a href="https://colab.research.google.com/github/alerodriargui/labs/blob/main/11_ResNET/ResNet_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
<a target="_blank" href="https://colab.research.google.com/github/bmalcover/aa_2425/blob/main/11_ResNET/ResNet_Custom.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

In [1]:
import os

import torch
from torch import nn
import torch.optim as optim
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import cv2

# Introducció

En aquesta pràctica treballarem amb l'arquitectura ResNet (Residual Network), centrant-nos en el concepte clau dels blocs residuals, que ja hem explorat a classe. Recordem breument la principal innovació d'aquestes: el bloc residual. Aquest introdueix una connexió directa (*shortcut connection*) entre l'entrada i la sortida d'un grup de capes, tal com es representa a continuació:

$$
\mathbf{y} = \mathcal{F}(\mathbf{x}, \{W_i\}) + \mathbf{x}
$$

On:
- $\mathbf{x}$ és l'entrada del bloc.
- $\mathcal{F}(\mathbf{x}, \{W_i\})$ és la transformació no lineal aplicada per les capes internes (normalment convolucions, normalització i ReLU).
- $\mathbf{y}$ és la sortida, que combina l'entrada original xx amb la transformació.

Aquest simple però efectiu mecanisme permet que el model "aprengui" només la diferència (o residual) entre l'entrada i la sortida esperada, en lloc de l'objectiu complet. Això facilita l'entrenament i redueix el risc de degradació del rendiment en xarxes profundes.

A classe hem vist que aquests blocs residuals són la base de diferents variants de ResNet, com ara ResNet-18, ResNet-50, etc., amb diferències en la profunditat i el nombre de blocs. Ara posarem en pràctica aquest coneixement treballant amb ResNets per a tasques de classificació d’imatges

# Cream mòduls amb Pytorch

En aquesta pràctica aprendrem a definir mòduls personalitzats en PyTorch utilitzant la classe base ``torch.nn.Module``. Aquesta classe permet encapsular les capes i operacions d’una xarxa neuronal, facilitant-ne la reutilització i el manteniment.

**Pasos per crear un mòdul en Pytorch:**
1. **Heretar de nn.Module**. Tots els mòduls personalitzats en PyTorch han de derivar de la classe base torch.nn.Module.
2. **Definir les capes en el constructor __init__.** Al constructor del mòdul (__init__), s’han d’inicialitzar les capes que s’utilitzaran, com ara convolucions, capes lineals o funcions d’activació.
3. **Implementar la funció forward.** Aquesta funció defineix com flueixen les dades a través del mòdul. Aquí s’apliquen les capes definides al constructor de manera seqüencial o segons sigui necessari.


## Cream un bloc residual

**Revisar sessió teòrica**

El nostre bloc residual tendrà dues capes convolucionals, batch norm i emprarà ReLU.

In [2]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()

        # Primera capa de convolución
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            padding='same',
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Segunda capa de convolución
        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            padding='same',
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Capa para la conexión de atajo (conv 1x1 si los canales de entrada/salida son diferentes)
        self.skip_connection = nn.Identity()
        if in_channels != out_channels:
            self.skip_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        # Ruta principal
        residual = x  # Guardar la entrada para la conexión de atajo
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)

        # Conexión de atajo y salida
        residual = self.skip_connection(residual)  # Ajustar la entrada si es necesario
        x += residual  # Sumar la conexión de atajo
        x = self.relu(x)  # Aplicar ReLU después de la suma

        return x

## Una VGG16 residual

Una vegada el tenim implementat farem dos models:
1. Model VGG16 **normal**. Un model com el que vàrem veure la setmana passada sense preentranement.
2. Model VGG16 **ampliat**. Heu d'afegir a una VGG16 dos blocs residuals al final de l'extractor de característiques. Per fer-ho s'ha d'emprar la mateixa estratègia que heu vist a les sessions anteriors per fer fine-tunning.

Entrena ambdós models amb el conjunt de dades [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), i contesta a la següent pregunta:

- És el mateix resultat una xarxa que l'altra? Si és així o no diguès per què?

In [3]:
from torchvision import models

class VGG16WithResidual(nn.Module):
    def __init__(self):
        super().__init__()

        # Cargar VGG16 preentrenada (sin la última capa fully connected)
        self.vgg16 = models.vgg16(pretrained=False)
        self.vgg16.classifier = nn.Identity()  # Eliminar la capa fully connected final

        # Añadir dos bloques residuales al final del extractor de características
        self.residual_block1 = ResidualBlock(in_channels=512, out_channels=512)
        self.residual_block2 = ResidualBlock(in_channels=512, out_channels=512)

        # Nueva capa fully connected para clasificación
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 10)  # Suponiendo 10 clases (como en Fashion MNIST)
        )

    def forward(self, x):
        x = self.vgg16.features(x)  # Pasar por el extractor de características VGG16
        x = self.residual_block1(x)  # Primer bloque residual
        x = self.residual_block2(x)  # Segundo bloque residual
        x = self.classifier(x)       # Clasificador final
        return x

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformaciones de datos para convertir a 3 canales
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convierte 1 canal a 3 canales
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalización para 3 canales
])

# Cargar Fashion MNIST con la nueva transformación
train_dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='data', train=False, transform=transform, download=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inicializar modelos y optimizadores
model_vgg16 = models.vgg16(pretrained=False, num_classes=10)
model_vgg16_with_residual = VGG16WithResidual()




# Entrenamiento
def train_model(model, train_loader, epochs=5):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for images, labels in tqdm(train_loader):
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
    return model

# Entrenar ambos modelos
train_model(model_vgg16, train_loader)
train_model(model_vgg16_with_residual, train_loader)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


  0%|          | 0/938 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# Evaluar ambos modelos
print("Evaluación VGG16:")
evaluate_model(model_vgg16, test_loader)

print("\nEvaluación VGG16 con bloques residuales:")
evaluate_model(model_vgg16_with_residual, test_loader)
